In [151]:
#importing the dependencies
import pandas as pd
import numpy as np
import time
import datetime
import os
from datetime import datetime as dt
from glob import glob

In [299]:
df = pd.read_csv(r'D:\Shyam\Documents\MUSER\Analysis\Users Datasets\muser-user-sayla.csv',encoding='utf-8-sig')
df.head(5)

,user_id,record_id,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_nano_time,event_seek_position_in_milliseconds,event_start_time,event_elapsed_time,song_album_id,...,device_info_device_model,device_info_google_play_services_app,device_info_google_play_services_lib,device_info_ignoring_battery_optimizations,device_info_power_save_mode_enabled,device_info_sdk_version,device_info_sdk_version_int,device_info_talk_back_enabled,device_info_timestamp,event_current_time_in_iso_utc
0,EPQKvHaHiJgLKa5W2REgB1dWRss1,1-da97964e-deba-4985-b113-3611bd8e183d,PLAY,NaN,1.635530e+12,1.084660e+13,0.0,NaN,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-29T17:40:27.514Z
1,EPQKvHaHiJgLKa5W2REgB1dWRss1,12-b1384535-672f-4d20-afa3-4005359cded6,PLAY,NaN,1.635530e+12,1.088510e+13,0.0,NaN,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-29T17:41:06.042Z
2,EPQKvHaHiJgLKa5W2REgB1dWRss1,13-18febb20-8d95-461d-9f02-998059553e03,PAUSE,NaN,1.635530e+12,1.089180e+13,6512.0,NaN,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-29T17:41:12.767Z
3,EPQKvHaHiJgLKa5W2REgB1dWRss1,15-bc9088da-8ede-42e0-b039-51ab43e28788,PLAY,NaN,1.635530e+12,1.090450e+13,0.0,NaN,NaN,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-29T17:41:25.471Z
4,EPQKvHaHiJgLKa5W2REgB1dWRss1,16-a0014ed5-563d-4cd2-8af0-fd01866ac35d,PLAY,NaN,1.635530e+12,1.091060e+13,0.0,NaN,NaN,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-29T17:41:31.557Z


In [300]:
#sorting the current timestamp in ascending order
df.sort_values('event_current_time_in_milliseconds', ascending=True, inplace=True)

In [301]:
# changing date type from utc iso to different date type
df['event_current_time_other'] = pd.to_datetime(df['event_current_time_in_iso_utc'])

In [302]:
#adding a new column event_current_time_ms_to_date to the csv 
df['event_current_time_ms_to_date'] = pd.to_datetime(df['event_current_time_in_milliseconds'], unit='ms')

In [303]:
#creating column- elapsed_time_in_min 
df['elapsed_time_in_min'] = round(df['event_seek_position_in_milliseconds']/60000,4)
#df.dtypes

In [304]:
#converting song_last_played to current timestamp
df['song_last_played_date'] = pd.to_datetime(df['song_last_played'], unit='ms')

In [305]:
#remove the empty valued audio_volume_data column
df = df.dropna(axis=0, subset=['audio_volume_data'])
df.shape

(27, 61)

In [306]:
#function to extract minimum volume from an array of audio_volume_data
def pull_vol_min(i):
    a,b,c,d= i.split(',')
    return c

In [307]:
df['min_volume'] =df['audio_volume_data'].apply(pull_vol_min)
df['min_volume'] = df['min_volume'].str.split('=').str[1]

In [308]:
#function to extract current volume from an array of audio_volume_data
def pull_vol_curr(i):
    a,b,c,d,e= i.split()
    start = b.find('[')
    end = b.find(',')
    substring = b[start+1:end]
    return substring

In [309]:
df['curent_volume'] = df.audio_volume_data.apply(pull_vol_curr)
df['curent_volume'] = df['curent_volume'].str.split('=').str[1]

In [310]:
#filtering out the records that are greater than the minimum volume value
df= df[ df['curent_volume'] > df['min_volume'] ]

In [311]:
# Create new variable curr_time to change the current time into milliseconds
curr_time = pd.to_datetime(df['event_current_time_in_iso_utc'])

# Create new column curr_time_in_ms
df['curr_time_in_ms'] = pd.to_datetime(curr_time, unit='ms')

In [312]:
# convert the new column to display milliseconds
df['curr_time_in_ms'] = df['curr_time_in_ms'].astype(np.int64) / int(1e6)

In [313]:
#creating column- current time in minutes 
df['curr_time_in_min'] = round(df['curr_time_in_ms']/60000,4)

In [314]:
# shorten the final file by only keeping required columns in the final csv file
user_behavior_data = df[["user_id", "record_id", "song_id", "song_name", "player_event_type", "ui_event_type",  "event_current_time_in_milliseconds", "event_seek_position_in_milliseconds", "song_album_name", "song_artist_name", "song_duration", "song_name", "audio_volume_data","event_current_time_ms_to_date", "event_current_time_in_iso_utc", "event_current_time_other", "min_volume", "curent_volume", "elapsed_time_in_min","song_last_played_date", "song_play_count"]]

In [315]:
user_behavior_data.head()

,user_id,record_id,song_id,song_name,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_seek_position_in_milliseconds,song_album_name,song_artist_name,...,song_name,audio_volume_data,event_current_time_ms_to_date,event_current_time_in_iso_utc,event_current_time_other,min_volume,curent_volume,elapsed_time_in_min,song_last_played_date,song_play_count
0,EPQKvHaHiJgLKa5W2REgB1dWRss1,1-da97964e-deba-4985-b113-3611bd8e183d,873.0,Hello,PLAY,NaN,1.635530e+12,0.0,25,Adele,...,Hello,"VolumeData [currentVolumeLevel=1, volumeMax=15...",2021-10-29 17:53:20,2021-10-29T17:40:27.514Z,2021-10-29 17:40:27.514000+00:00,0,1,0.0,1970-01-01,0.0
24,EPQKvHaHiJgLKa5W2REgB1dWRss1,6-29607767-72b1-4363-afcf-95c2a1545217,NaN,NaN,NaN,VOLUME_UP,1.635530e+12,0.0,NaN,NaN,...,NaN,"VolumeData [currentVolumeLevel=6, volumeMax=15...",2021-10-29 17:53:20,2021-10-29T17:40:40.807Z,2021-10-29 17:40:40.807000+00:00,0,6,0.0,NaT,NaN
23,EPQKvHaHiJgLKa5W2REgB1dWRss1,5-dd7c1645-75ef-4ba5-8798-84289fbd6e9d,NaN,NaN,NaN,VOLUME_UP,1.635530e+12,0.0,NaN,NaN,...,NaN,"VolumeData [currentVolumeLevel=5, volumeMax=15...",2021-10-29 17:53:20,2021-10-29T17:40:40.453Z,2021-10-29 17:40:40.453000+00:00,0,5,0.0,NaT,NaN
22,EPQKvHaHiJgLKa5W2REgB1dWRss1,4-42b65077-4258-42b2-b692-f11d3a9eeb90,NaN,NaN,NaN,VOLUME_UP,1.635530e+12,0.0,NaN,NaN,...,NaN,"VolumeData [currentVolumeLevel=4, volumeMax=15...",2021-10-29 17:53:20,2021-10-29T17:40:40.160Z,2021-10-29 17:40:40.160000+00:00,0,4,0.0,NaT,NaN
21,EPQKvHaHiJgLKa5W2REgB1dWRss1,3-87abb09c-9ed5-4530-924c-8b39978e4eaf,NaN,NaN,NaN,VOLUME_UP,1.635530e+12,0.0,NaN,NaN,...,NaN,"VolumeData [currentVolumeLevel=3, volumeMax=15...",2021-10-29 17:53:20,2021-10-29T17:40:40.087Z,2021-10-29 17:40:40.087000+00:00,0,3,0.0,NaT,NaN


In [316]:
## Original Code
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  listening_time = 0
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if event_type != "PLAY" and user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY": #this is covering same song pause-play
        listening_time = trigger_time - user_behavior_data.loc[idx-1, "event_current_time_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)
    if (event_type == "PAUSE" or event_type == "NEXT" or event_type == "PREV") and (user_behavior_data.loc[idx-1, "player_event_type"] == "SEEK" or user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY"):#when idx-1 is play and idx is either play or next or prev
        listening_time = trigger_time - user_behavior_data.loc[idx-1, "event_current_time_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)

C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [317]:
# USER EVENT

# Duration for half song duration
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  ui_event = user_behavior_data.loc[idx, "ui_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  song_duration_time = user_behavior_data.loc[idx, "song_duration"]
  event_seek = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  user_half_song = 0  
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if ui_event == "PLAY" and user_behavior_data.loc[idx, "ui_event_type"] == "PLAY":
        if user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] >= user_behavior_data.loc[idx, "song_duration"]/2:
            user_half_song = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
            user_behavior_data.loc[idx, "user_half_song"] = user_half_song/(1000*60)
    if (ui_event == "PAUSE" or ui_event == "NEXT" or ui_event == "PREV") and (user_behavior_data.loc[idx-1, "ui_event_type"] == "SEEK" or user_behavior_data.loc[idx-1, "ui_event_type"] == "PLAY"):
        if user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] >= user_behavior_data.loc[idx, "song_duration"]/2:
            user_half_song = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
            user_behavior_data.loc[idx, "user_half_song"] = user_half_song/(1000*60)

In [318]:
# PLAYER EVENT TYPE

# TRIAL ONE
# Duration for half song duration
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  ui_event = user_behavior_data.loc[idx, "ui_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  song_duration_time = user_behavior_data.loc[idx, "song_duration"]
  event_seek = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  player_half_song = 0  
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if event_type != "PLAY" and user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY":
        if user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] >= user_behavior_data.loc[idx, "song_duration"]/2:
            player_half_song = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
            user_behavior_data.loc[idx, "player_half_song"] = player_half_song/(1000*60)
    if (event_type == "PAUSE" or event_type == "NEXT" or event_type == "PREV") and (user_behavior_data.loc[idx-1, "player_event_type"] == "SEEK" or user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY"):
        if user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] >= user_behavior_data.loc[idx, "song_duration"]/2:
            player_half_song = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
            user_behavior_data.loc[idx, "player_half_song"] = player_half_song/(1000*60)

In [319]:
#path = r'A:\MUSER\data\cleaned_data\mergedFiles\dummy\analysis_Code'
path = r'D:\Shyam\Documents\MUSER\Analysis\User Cleaned Data'

user_behavior_data.to_csv(os.path.join(path,'6_muser_user_behavior_data.csv'), index=False, encoding='utf-8-sig')